# final setup
I’ve tried many alternative, different views but I believe there is just not more information in the limited data set. Please note that we only have a sample of the E_mails, meaning that the input into the algo’s is sub-optimal.

Given that different experimental set-up don’t result in a significant different picture, I chose to use the leasts ambiguous setup and to follow the labels from Creamer (1-on-1) using detailed labels avoiding biased in the aggregation of labels.

In [ ]:
# run this only once!!
import os
os. chdir("../..")
os.getcwd()

In [ ]:
import networkx as nx
from graphcase_experiments.algos.GraphCaseWrapper import GraphCaseWrapper
from graphcase_experiments.tools.embedding_plotter import plot_embedding2, plotly_embedding
from graphcase_experiments.tools.calculate_embed import calculate_graphcase_embedding
from graphcase_experiments.graphs.enron.mail_reader import spark
import matplotlib.pyplot as plt
import pydot
import pandas as pd
import tensorflow as tf
import pyspark.sql.functions as F

In [ ]:
# load graph
SOURCE_PATH = 'graphcase_experiments/graphs/enron/data/'
G = nx.read_gpickle(SOURCE_PATH + 'enron_sub_graph4.pickle')

In [ ]:
G.number_of_edges()

In [ ]:
labels = []
for n, d in G.nodes(data=True):
    labels.append(d['label'])

pdf = pd.DataFrame(labels, columns=['labels'] )
pdf.groupby('labels')['labels'].count()

In [ ]:
# alternative mapping - smry lbls
from graphcase_experiments.tools.calculate_embed import calculate_graphcase_embedding
from graphcase_experiments.algos.GraphCaseWrapper import GraphCaseWrapper

_, tbl = calculate_graphcase_embedding(
            G, 'graphcase_experiments/data/enron/', params=GraphCaseWrapper.ENRON_PARAMS, verbose=True, algo=GraphCaseWrapper
        )
plotly_embedding(tbl)

In [ ]:
# get the confussion matrix
from sklearn.metrics import ConfusionMatrixDisplay
from graphcase_experiments.experiments.ring_exp import classify_svm
svm_res = classify_svm(tbl, test_size=0.5)
ConfusionMatrixDisplay.from_predictions(tbl['label_id'], svm_res['pred_labels'])

In [ ]:
tbl[['label', 'label_id']].drop_duplicates()

## gridsearch

In [ ]:
# gridsearch
from graphcase_experiments.tools.gridsearch import grid_search_graphcase
def search_params(trial):
    return {
        'learning_rate': trial.suggest_float("learning_rate", 1e-5, 1e-4, log=True),
        'act': trial.suggest_categorical("act", ['relu', 'sigmoid', 'identity']),
        'useBN': trial.suggest_categorical("useBN", [True]),
        'dropout': trial.suggest_float("dropout", 0.1, 0.2),
        'support_size': trial.suggest_int("support_size", 7, 10),
        'dims': trial.suggest_int("dims", 32, 128)
    }
FIXED_PARAMS = {
        'batch_size': 30,
        'hub0_feature_with_neighb_dim': 128,
        'verbose': False,
        'seed': 1,
        'encoder_labels': ['attr_received_size', 'attr_cnt_to', 'attr_cnt_cc', 'attr_sent_size', 'attr_cnt_send'],
        'epochs': 10,  #1000,
        'trials': 3  #200
    }

PATH = 'graphcase_experiments/data/enron/'

grid_search_res = grid_search_graphcase(G, PATH, [search_params, FIXED_PARAMS])


In [ ]:
grid_search_res[1]

## calculate performance

In [ ]:
# alternative mapping - detailed lbls
from graphcase_experiments.algos.GraphCaseWrapper import GraphCaseWrapper
from graphcase_experiments.experiments.enron_comp import calc_enron_performance

ind, res = calc_enron_performance(G=G, test_size=0.5, runs=10)

In [ ]:
res

In [ ]:
# import pickle
# filename = 'graphcase_experiments/data/results/reallife/enron_results.pickle'
# with open(filename, 'wb') as handle:
#     pickle.dump(res, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
res  # met 0.1 default weight on dummy

In [ ]:
res

In [ ]:
#na adding class weight in the svm
res

In [ ]:
res

# summary labels


In [ ]:
main_label = {
    'ceos-president': 'senior_mngt',
    'managing director': 'senior_mngt',
    'executive vp': 'senior_mngt',
    'vice president': 'senior_mngt',
    'director': 'middle_mngt',
    'manager': 'middle_mngt',
    'trader': 'trader',
    'specialist': 'employee',
    'employee': 'employee',
    'attorney-legal asst.': 'employee',
}
G2 = G.copy()
relabel =  {n: main_label[a.lower()] for n,a in G.nodes(data='label')}
nx.set_node_attributes(G2, relabel, 'label')

In [ ]:
# alternative mapping - detailed lbls
from graphcase_experiments.experiments.enron_comp import calc_enron_performance
ind2, res2 = calc_enron_performance(G=G2, test_size=0.5, runs=5, algos=[GraphCaseWrapper])

In [ ]:
res2

In [ ]:
# using class weight in SVM
res2

In [ ]:
res2